<a href="https://colab.research.google.com/github/nakamura196/ndl_ocr/blob/main/NDL%E5%8F%A4%E5%85%B8%E7%B1%8DOCR%E3%81%AE%E5%AE%9F%E8%A1%8C%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NDL古典籍OCRの実行例：Google Driveを用いた画像の入力と結果の保存

本ノートブックのライセンス： <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />

<br/>

NDL古典籍OCRの説明: https://github.com/ndl-lab/ndlkotenocr_cli

<br/>

本ノートブックでは、Google Driveに認識結果を出力します。

<br/>

## 参考にしたノートブック

@blue0620 さんが作成したノートブック

https://github.com/blue0620/NDLkotenOCR-GoogleColabVersion/blob/main/NDLkotensekiOCR_googlecolabversion.ipynb

<br/>

## 更新内容

- 2023-06-28
 - 一部のオプションに、検索可能なPDFの出力機能を追加しました。 <!-- -->
- 2023-06-15
 - Google Colabで実行できない不具合を解消しました。プログラムを改修してくださったNDLおよび@blue0620さんに感謝申し上げます。
- 2023-04-12
 - OCR結果が表示されない不具合を修正しました。
- 2023-02-15
 - アノテーションがずれて表示される不具合を修正しました。
- 2023-01-25
 - ノートブックを公開しました。

<br/><br/><br/><br/>

## 1.初期セットアップ

4分ほど時間がかかります。初回のみ実行が必要です。

In [ ]:
#@title セットアップ

%cd /content/

# Google Drive関連
from google.colab import drive
drive.mount('/content/drive/')

# Googleドライブのパスを取得
!pip install kora
from kora.xattr import get_id

def message(path):
  print("\n以下に出力しました。\nhttps://drive.google.com/drive/folders/{}".format(get_id(path)))

# OCR関連のセットアップ
!git clone --recursive https://github.com/ndl-lab/ndlkotenocr_cli
PROJECT_DIR="/content/ndlkotenocr_cli"
# !pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118
!pip install mmcv==2.0.0 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html
# !pip install -r {PROJECT_DIR}/requirements.txt

!pip install mmdet
!pip install mmcls==1.0.0rc6
!pip install transformers

%cd {PROJECT_DIR}
!wget -nc https://lab.ndl.go.jp/dataset/ndlkotensekiocr/trocr/models.zip -P ./src/text_kotenseki_recognition/
!wget -nc https://lab.ndl.go.jp/dataset/ndlkotensekiocr/layoutmodel/models.zip -P ./src/ndl_kotenseki_layout/
!unzip -o ./src/text_kotenseki_recognition/models.zip -d ./src/text_kotenseki_recognition/
!unzip -o ./src/ndl_kotenseki_layout/models.zip -d ./src/ndl_kotenseki_layout/
%cd /content/

# 出力結果のクリア
from IPython.display import clear_output

# その他
!pip install ndl-ocr-tools

# 処理関連

import time
import datetime
import pytz
from pathlib import Path
import os

def create_settings(output_dir):
  output_dir = str(Path(output_dir))

  task_id = output_dir.split("/")[-1]

  if os.path.exists(output_dir):
    time_str = datetime.datetime.now(pytz.timezone('Asia/Tokyo')).strftime('%Y%m%d%H%M%S')
    task_id += "_" + time_str

  output_dir_mod = os.path.dirname(output_dir) + "/" + task_id
  # output_dir_mod

  tmp_dir = f"/content/tmp/{task_id}"
  img_tmp_dir = f"{tmp_dir}/img"

  # ディレクトリを作成
  os.makedirs(img_tmp_dir, exist_ok=True)

  os.makedirs(os.path.dirname(output_dir_mod), exist_ok=True) # 注意。上位フォルダを作成する。

  return {
      # "img_tmp_dir": img_tmp_dir,
      "input_dir": tmp_dir,
      "output_dir": output_dir_mod
  }

def inf(config, is_clear_output=True):
  tmp_dir = config["input_dir"]
  output_dir_mod = config["output_dir"]
  %cd {PROJECT_DIR}
  !python main.py infer {tmp_dir} {output_dir_mod} -s s
  %cd /content

  if is_clear_output:
    clear_output()

  message(output_dir_mod)

################# 認識結果の閲覧　#################

# サーバの起動
PORT = 8001
%cd /content
!nohup python3 -m http.server $PORT > server.log 2>&1 &
!wget https://mirador.cultural.jp/index.html
from google.colab import output

# 独自ツール
!pip install ndl_ocr_tools

from ndl_ocr_tools.api import Client

def showMirador(urls, config, check_iiif=True):
  # %cd /content

  task_id = config["output_dir"].split("/")[-1]

  output_final_dir = f'{config["output_dir"]}/{task_id}'

  json_path =  f'{output_final_dir}/json'
  img_dir = f'{config["input_dir"]}/img'
  output_path = f"{output_final_dir}/manifest.json"
  Client.createIIIFManifest3(output_path, urls, img_dir, json_path, check_iiif=check_iiif)

  print("\n認識結果は以下のURLでご確認ください。")
  output.serve_kernel_port_as_window(PORT, path=f'/?manifest={output_path.replace("/content", "")}&annotationState=1')

  return output_path

# pdf
!pip install ocr-iiif-tools==0.0.11

from ocr_iiif_tools.pdf import *
pdfClient = PdfClient()

def createPdf(manifest_path, config):
  output_pdf_dir = os.path.dirname(manifest_path)
  output_path = f"{output_pdf_dir}/document.pdf"
  pdfClient.convert_iiif2pdf(output_path, iiif_manifest_path=manifest_path, image_download_dir=config["input_dir"] + "/img")

clear_output()

<br/><br/><br/><br/>

## 2.設定

以下、入力方式によって適切なものを選んでください。

- 画像
  - [単一の画像ファイルのURLを指定する場合](#scrollTo=79RZXnYuuXSm)
  - [単一の画像ファイルをアップロードする場合](#scrollTo=RUnc5ujkGIG-)
  - [複数の既にダウンロード済みの画像ファイルを対象にする場合](#scrollTo=elmieBAGH9Bc)
- IIIF
  - [IIIFマニフェストファイルのURLを指定する場合](#scrollTo=Jh0O2Da_0Snv)

<br/><br/><br/><br/>

## 画像

<br/>

### 単一の画像ファイルのURLを指定する場合

- URL: 画像ファイルのURL
- 出力フォルダ: 出力するフォルダへのパス

入力サンプル：「源氏物語」（国立国会図書館所蔵）

In [ ]:
#@title 設定

URL = "https://dl.ndl.go.jp/api/iiif/2585098/R0000003/full/full/0/default.jpg" #@param {type:"string"}
出力フォルダ = "/content/drive/MyDrive/ndl_kotenseki_ocr/output/image_url" #@param {type:"string"}

# 設定の取得
config = create_settings(出力フォルダ)

# 画像のダウンロード
from urllib import request
request.urlretrieve(URL, config["input_dir"] + "/img/0001.jpg")

# 推論の実行
inf(config)

# 認識結果の表示
manifest_path = showMirador([URL], config, check_iiif=False)

# pdfの作成
createPdf(manifest_path, config)

<br/><br/><br/>

### 単一の画像ファイルをアップロードする場合

- 出力フォルダ: 出力するフォルダへのパス

以下の設定の再生ボタンを押すと、ファイルのアップロードフォームが表示されます。

In [ ]:
#@title 設定

出力フォルダ = "/content/drive/MyDrive/ndl_kotenseki_ocr/output/image_local" #@param {type:"string"}

from google.colab import files
uploaded = files.upload()
input_file = next(iter(uploaded))

# 設定の取得
config = create_settings(出力フォルダ)

# 画像のコピー
!mv {input_file} {config["input_dir"] + "/img/0001.jpg"}

# 推論の実行
inf(config)

<br/><br/><br/>

### 複数の既にダウンロード済みの画像ファイルを対象にする場合

- 入力フォルダ: 入力するフォルダのパス
  - 指定したフォルダの下にimgフォルダを用意し、その中に画像を格納してください。
- 出力フォルダ: 出力するフォルダへのパス

In [ ]:
#@title 設定

入力フォルダ = "/content/drive/MyDrive/ndl_kotenseki_ocr/input" #@param {type:"string"}
出力フォルダ = "/content/drive/MyDrive/ndl_kotenseki_ocr/output/image_single" #@param {type:"string"}

# 設定の取得
config = create_settings(出力フォルダ)

# 入力フォルダの更新
config["input_dir"] = 入力フォルダ

# 推論の実行
inf(config)

<br/><br/><br/><br/>

## IIIF

<br/>

### IIIFマニフェストファイルのURLを指定する場合

- IIIFマニフェストファイルのURL: IIIFマニフェストファイルのURL
- 出力フォルダ: 出力するフォルダへのパス
- 開始コマ数: 処理を開始するコマ。デフォルトは1。
- 終了コマ数: 処理を終了するコマ。デフォルトは5。-1にするとすべて。
- 画像ダウンロードの間隔_秒数: 画像ダウンロードの間隔（秒数）

入力サンプル：「源氏物語」（国立国会図書館所蔵）

In [ ]:
#@title 設定
IIIFマニフェストファイルのURL = "https://dl.ndl.go.jp/api/iiif/2585098/manifest.json"#@param {type:"string"}

出力フォルダ = "/content/drive/MyDrive/ndl_kotenseki_ocr/output/iiif" #@param {type:"string"}

開始コマ数 =   1 #@param {type:"number"}
終了コマ数 =   5 #@param {type:"number"}
画像ダウンロードの間隔_秒数 =   1 #@param {type:"number"}

# 設定の取得
config = create_settings(出力フォルダ)

# 画像のダウンロード
urls = Client.downloadImagesFromManifest(IIIFマニフェストファイルのURL, config["input_dir"] + "/img", pos_start=開始コマ数, pos_end=終了コマ数, sleep_time=画像ダウンロードの間隔_秒数)

# 推論の実行
inf(config)

# 認識結果
manifest_path = showMirador(urls, config)

# pdfの作成
createPdf(manifest_path, config)

# その他

In [ ]:
import torch
print(torch.__version__)
!python3 -V